In [1]:
"""install libraries"""
!pip install gym gym[atari]

"""Import libraries"""
import gym
import numpy as np
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
from time import sleep
from matplotlib import animation

In [36]:
env = gym.make("ALE/Pitfall-v5", render_mode="rgb_array").env
state, _ = env.reset()
n_actions = env.action_space.n


# Print dimensions of state and action space
print("State space: {}".format(env.observation_space.shape))
print(f"Action space: {env.unwrapped.get_action_meanings()}")

# Sample random action
action = env.action_space.start
next_state, reward, done, _, _ = env.step(action)

# Print output
print("State: {}".format(state))
print("Action: {}".format(action))
print("Reward: {}".format(reward))

# Render and plot an environment frame
frame = env.render()
plt.imshow(frame)
plt.axis("off")
plt.show()

State space: (210, 160, 3)
Action space: ['NOOP', 'FIRE', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'UPRIGHT', 'UPLEFT', 'DOWNRIGHT', 'DOWNLEFT', 'UPFIRE', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE', 'UPRIGHTFIRE', 'UPLEFTFIRE', 'DOWNRIGHTFIRE', 'DOWNLEFTFIRE']


TypeError: list indices must be integers or slices, not builtin_function_or_method